In [2]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.config.id2label = {0: "location", 1: "people name"}
text = "My name is Aman. I am a student."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)


In [15]:
with torch.no_grad():
    outputs = model(**inputs)

predicted_labels_indices = torch.argmax(outputs.logits, dim=2)


In [19]:
# Convert token IDs to labels
label_names = [model.config.id2label[label_id.item()] for label_id in predicted_labels_indices]

# Print token and corresponding label name
decoded_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
for token, label_name in zip(decoded_tokens, label_names):
    if '##' in token:
        # Handle subword tokens by appending them to the previous token
        print(f"Token: {decoded_tokens[-1]}{token.replace('##', '')}\tPredicted Label: {label_name}")
    else:
        print(f"Token: {token}\tPredicted Label: {label_name}")

RuntimeError: a Tensor with 12 elements cannot be converted to Scalar

In [34]:
from transformers import BertTokenizer, BertForTokenClassification

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')

# Modify the id2label mapping in the model's configuration
model.config.id2label = {0: "location", 1: "people name",2:"organization"}  # Modify this mapping according to your labels

# Define an example input in Nepali
text = "मेरो नाम अमन हो। म एक विद्यार्थी हूँ।"

# Tokenize and encode input
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

inputs
# # Run inference
# with torch.no_grad():
#     outputs = model(**inputs)

# # Get predicted labels (integer indices)
# predicted_label_indices = torch.argmax(outputs.logits, dim=2)[0]

# # Convert label indices to label names using modified model's configuration
# label_names = [model.config.id2label[label_id.item()] for label_id in predicted_label_indices]

# # Print token and corresponding label name
# decoded_tokens = []
# for token, label_name in zip(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]), label_names):
#     if '##' in token and len(decoded_tokens) > 0:
#         decoded_tokens[-1] += token.replace('##', '')
#     else:
#         decoded_tokens.append(token.replace('##', ''))
    
# for token, label_name in zip(decoded_tokens, label_names):
#     print(f"Token: {token}\tPredicted Label: {label_name}")


c:\Users\Rabin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[   101,  40265, 101841,  21645,    851,  62752,  13220,    920,    889,
          11186,  55190,  97110,  19885,  69002,  10914,    899,  15778,  28462,
            920,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## First Phase

In [1]:
import pandas as pd

# Load labeled data from CSV
data = pd.read_csv('chunk_2.csv')

# Separate text and labels
texts = data['Column1'].tolist()
labels = data['Column2'].tolist()
print(labels)
# Convert labels to integer indices (assuming you have label-to-index mapping)
label_to_index = {'0': 0, 'B-Person': 1,'B-Organization':2, 'I-Organization':3, 'B-Location':4, 'B-Event':5, 'I-Event':6,'0':7}# Update this mapping

default_index = 8
label_indices = [label_to_index.get(label, default_index) for label in labels]


['O', 'O', nan, 'B-Organization', 'I-Organization', 'I-Organization', 'I-Organization', 'O', 'O', 'O', 'O', 'B-Person', 'O', 'B-Event', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Date', 'I-Date', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Date', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', nan, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Person', 'I-Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', nan, 'O', 'O', 'O', 'B-Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', nan, 'O', 'O', 'O', 'O', nan, 'O', 'O', 'O', 'O', 'O', nan, 'O', 'B-Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', nan, 'B-Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', nan, 'B-Date', 'I-Date', 'I-Date', 'I-Date', 'O', 'B-Organization', 'O', 'O', 'O',

In [13]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification
import torch
tokenizer = BertForTokenClassification.from_pretrained('bert-base-uncased')
# Tokenize and encode text
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, padding='max_length', max_length=128, truncation=True) for text in texts]
# Define a custom PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, tokenized_texts, label_indices):
        self.tokenized_texts = tokenized_texts
        self.label_indices = label_indices
    
    def __len__(self):
        return len(self.tokenized_texts['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'labels': self.label_indices[idx]
        }

# Create a DataLoader
batch_size = 32
train_dataset = CustomDataset(tokenized_texts, label_indices)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


c:\Users\Rabin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rabin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are 

TypeError: BertForTokenClassification.forward() got an unexpected keyword argument 'return_tensors'

In [61]:

# Fine tuning 
from transformers import BertForTokenClassification, AdamW
import torch

# Load pre-trained BERT model for token classification
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(labels_to_index))

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask'],
            'labels': batch['labels']
        }
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f}')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Expected input batch_size (20) to match target batch_size (1).

In [14]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load labeled data from CSV
data = pd.read_csv('chunk_2.csv')

# Separate text and labels
texts = data['Column1'].tolist()
bio_labels = data['Column2'].tolist()

# Create a tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')

# Tokenize and encode text for each entry
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, padding='max_length', max_length=128, truncation=True) for text in texts]

# Convert BIO labels to label indices
label_to_index = {'0': 0, 'B-Person': 1,'B-Organization':2, 'I-Organization':3, 'B-Location':4, 'B-Event':5, 'I-Event':6,'0':7}  # Update this mapping
label_indices = []

for bio_labels in bio_labels:
    indices = [label_to_index[label] for label in bio_labels.split()]
    label_indices.append(indices)

# Convert the flattened list to a tensor
label_tensor = torch.tensor(label_indices, dtype=torch.long)

# Reshape the label tensor to match the token count
num_tokens = max(len(tokenized_text['input_ids']) for tokenized_text in tokenized_texts)
label_tensor = label_tensor.view(-1, num_tokens)

# Define a custom PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, tokenized_texts, label_tensor):
        self.tokenized_texts = tokenized_texts
        self.label_tensor = label_tensor
    
    def __len__(self):
        return len(self.tokenized_texts)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_texts[idx]['input_ids'],
            'attention_mask': self.tokenized_texts[idx]['attention_mask'],
            'labels': self.label_tensor[idx]
        }

# Create a DataLoader
batch_size = 32
train_dataset = CustomDataset(tokenized_texts, label_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.